<h1>IVI - Traces Web et module selenium</h1>

# 1. Installation du module et webdrivers

Depuis le terminal (UNIX) ou invite de commande (Windows).

In [ ]:
pip install selenium

* [**selenium**](https://selenium-python.readthedocs.io/): Automatisation et manipulation de webdriver.

### Webdrivers
Télécharger le/les webdriver(s) désiré(s), correspondants aux versions de vos applications installées.
* [**Firefox**](https://github.com/mozilla/geckodriver/releases)
* [**Chrome**](https://sites.google.com/chromium.org/driver/)
* [Edge](https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/), [Safari](https://webkit.org/blog/6900/webdriver-support-in-safari-10/), [Opera](https://github.com/operasoftware/operachromiumdriver/releases), ect.

# 2. Script webdriver.py

## 2.1. Importation du module

In [1]:
from selenium import webdriver

## 2.2. Firefox (geckodriver)

### 2.2.1. Instanciation du webdriver

In [2]:
gecko_path = './webdrivers/geckodriver.exe'

In [3]:
driver = webdriver.Firefox(executable_path=gecko_path)

**gecko_path** est le chemin d'accès au programme du webdriver (à changer).

*Remarque*: Il est également possible d'ajouter les webdrivers directement aux variables d'environnement.

### 2.2.2. Configurer votre webdriver

De [nombreux paramètres](http://kb.mozillazine.org/About:config) peuvent être configurés lors de l'utilisation du webdriver.

In [4]:
options = webdriver.firefox.options.Options()

options.headless = False #masque la fenêtre: True pour la masquer, False pour l'afficher.
options.set_preference("dom.push.enabled", False) #bloque les popup de notifications

profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36") #définit un useragent

In [5]:
driver = webdriver.Firefox(firefox_profile=profile, executable_path=gecko_path, options=options)

## 2.4. La fonction get()

La  fonction get() permet de demander l'accès à une ressource web via son URL.

In [6]:
url = 'http://example.com/'
driver.get(url)

### Résultat
Le navigateur contrôlé génère la requête HTTP et interprète le code reçu.

### <span style="color: red;">Code client</span>
Chaine de caractères avec les données du code contenu dans le navigateur, après exécution du code serveur reçu.

In [7]:
client_code = driver.page_source

In [8]:
print(client_code)

<html><head>
    <title>Example Domain</title>

    <meta charset="utf-8">
    <meta http-equiv="Content-type" content="text/html; charset=utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domain is for use in illustr

### <span style="color: red;">Capture d'écran</span>

La fonction **driver.save_screenshot('chemin_du_fichier')** permet de faire une capture d'écran de la fenêtre du navigateur et l'enregistrer dans un fichier.

In [10]:
def screenshot(driver, outputPath, width=None):
        if width==None: 
            width = driver.execute_script('return document.documentElement.scrollWidth')
        
        heightMax = driver.execute_script('return document.documentElement.scrollHeight')
        
        driver.set_window_size(width, heightMax)
        driver.save_screenshot(outputPath)

*Remarque*: A l'aide des fonctions JavaScript il est possible d'obtenir la largeur et longueur maximale de la page avec respectivement "*return document.documentElement.scrollWidth*" et "*return document.documentElement.scrollHeight*".

**driver.set_window_size(width, height)** permet de définir la taille de la fenêtre.

In [11]:
screenshot(driver, 'test.png', 20)

*Remarque*: La capture complète de la page ne fonctionne qu'avec le mode "headless" dans la majorité des cas.

### <span style="color: red;">Code serveur</span>
Normalement selenium ne nous permet pas d'accéder directement à la réponse HTTP, elle est directement interprétée par le navigateur.

**Astuce**: l'utilisation du [schéma d'URI](https://fr.wikipedia.org/wiki/Sch%C3%A9ma_d%27URI) "view-source" supporté par certains navigateurs (dont Firefox et Chrome) permet d'afficher le code reçu du serveur.

In [ ]:
def serverCode(driver):
    url = driver.current_url
    driver.get('view-source:'+url)
    server_code = driver.find_element_by_xpath("//*").text
    driver.back()
    
    return server_code

**driver.current_url** retourne l'url actuelle du navigateur.

**driver.find_element_by_xpath("//*")** permet de séléctionner l'élément racine du document.

In [ ]:
server_code = serverCode(driver)
print(server_code)

*Remarque*: Attention exécuter la fonction change la page du navigateur. La fonction driver.back() permet de revenir à la page précédente.

## 2.5. Proxy TOR

Il faut préalablement ouvrir le port de communication pour TOR. Nous utiliserons dans l'exemple le port 9050.

### 2.5.1. Firefox

In [ ]:
profile = webdriver.FirefoxProfile()

profile.set_preference("network.proxy.type", 1)
profile.set_preference("network.proxy.socks", "127.0.0.1")
profile.set_preference("network.proxy.socks_port", 9050)
profile.set_preference("network.proxy.socks_version", 5)
profile.set_preference("network.proxy.socks_remote_dns", True)

## 2.6. Manipulation du navigateur

Exemple: Remplir un formulaire et l'envoyer


In [14]:
url = 'https://www.w3schools.com/html/html_forms.asp'
driver.get(url)

Sélection d'un élément (balise) par son attribut id. Enlever la valeur (clear) et envoi d'un texte (send_keys).


In [16]:
nom = "Démo"
driver.find_element_by_id('fname').clear()
driver.find_element_by_id('fname').send_keys(nom)

In [18]:
nom = "Robert"
driver.find_element_by_id('lname').clear()
driver.find_element_by_id('lname').send_keys(nom)

Soumettre le formulaire

In [19]:
driver.find_element_by_id('lname').submit()